In [1]:
import pandas as pd
import numpy as np
import warnings
from functions import metrics
import csv
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.exceptions import ConvergenceWarning
from sklearn.metrics import mean_squared_error
from functions.formatting import get_indices, get_subgroup_str

warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [4]:
omit_demographics = True 
use_gerryfair = False 

In [5]:
df = pd.read_csv('./data/preprocessed.csv')
df.drop(['umich_user_id', 'Unnamed: 0'], axis=1, inplace=True)

In [6]:
X = df.drop('completed', axis=1)
y = df['completed']

#### Calculate false positive rates for the given subgroup

This function takes in the subgroup to test for as a list of tuples. We use logistic regression and 10-fold cross validation to calculate the average FPR and AUC for the given subgroup. 

Returns a tuple containing FPR average, FPR standard deviation, AUC average, AUC standard deviation, and number of students in a given subgroup

#### Compute all subgroups

This function generates all possible subgroups given the list of protected columns

In [7]:
def compute_combo(cols, races, genders):
    retval = [[]]
    def recurse(i, curr):
        for r in races:
            opt3 = curr.copy()
            opt3.append((r, 1))
            if opt3 not in retval:
                retval.append(opt3)
        for g in genders:
            opt4 = curr.copy()
            opt4.append((g, 1))
            if opt4 not in retval:
                retval.append(opt4)
        for r in races:
            for g in genders:
                opt5 = curr.copy()
                opt5.append((r, 1))
                opt5.append((g, 1))
                if opt5 not in retval:
                    retval.append(opt5)
        for j in range(i, len(cols)):
            opt1 = curr.copy()
            opt1.append((cols[j], 0))

            opt2 = curr.copy()
            opt2.append((cols[j], 1))
            
            retval.append(opt1)
            retval.append(opt2)
            
            if j < len(cols):
                recurse(j+1, opt1.copy())
            if j < len(cols):
                recurse(j+1, opt2.copy())
        
    recurse(0, [])

    return retval

In [8]:
combos = compute_combo(['country_cd_US', 'bachelor_obtained'], ['white', 'black', 'asian', 'latinx', 'race_others', 'race_na'], ['male', 'female', 'gender_na', 'gender_other']) 

In [9]:
combos.sort(key=len)
combos

[[],
 [('white', 1)],
 [('black', 1)],
 [('asian', 1)],
 [('latinx', 1)],
 [('race_others', 1)],
 [('race_na', 1)],
 [('male', 1)],
 [('female', 1)],
 [('gender_na', 1)],
 [('gender_other', 1)],
 [('country_cd_US', 0)],
 [('country_cd_US', 1)],
 [('bachelor_obtained', 0)],
 [('bachelor_obtained', 1)],
 [('white', 1), ('male', 1)],
 [('white', 1), ('female', 1)],
 [('white', 1), ('gender_na', 1)],
 [('white', 1), ('gender_other', 1)],
 [('black', 1), ('male', 1)],
 [('black', 1), ('female', 1)],
 [('black', 1), ('gender_na', 1)],
 [('black', 1), ('gender_other', 1)],
 [('asian', 1), ('male', 1)],
 [('asian', 1), ('female', 1)],
 [('asian', 1), ('gender_na', 1)],
 [('asian', 1), ('gender_other', 1)],
 [('latinx', 1), ('male', 1)],
 [('latinx', 1), ('female', 1)],
 [('latinx', 1), ('gender_na', 1)],
 [('latinx', 1), ('gender_other', 1)],
 [('race_others', 1), ('male', 1)],
 [('race_others', 1), ('female', 1)],
 [('race_others', 1), ('gender_na', 1)],
 [('race_others', 1), ('gender_other',

In [10]:
groups = {}
for combo in combos:
    group_key = []
    for c in combo:
        group_key.append(c[0])
    if tuple(group_key) not in groups:
        groups[tuple(group_key)] = []
    groups[tuple(group_key)].append(combo)

# sort dict by length of key
groups = dict(sorted(groups.items(), key=lambda item: len(item[0])))

print(groups)


{(): [[]], ('white',): [[('white', 1)]], ('black',): [[('black', 1)]], ('asian',): [[('asian', 1)]], ('latinx',): [[('latinx', 1)]], ('race_others',): [[('race_others', 1)]], ('race_na',): [[('race_na', 1)]], ('male',): [[('male', 1)]], ('female',): [[('female', 1)]], ('gender_na',): [[('gender_na', 1)]], ('gender_other',): [[('gender_other', 1)]], ('country_cd_US',): [[('country_cd_US', 0)], [('country_cd_US', 1)]], ('bachelor_obtained',): [[('bachelor_obtained', 0)], [('bachelor_obtained', 1)]], ('white', 'male'): [[('white', 1), ('male', 1)]], ('white', 'female'): [[('white', 1), ('female', 1)]], ('white', 'gender_na'): [[('white', 1), ('gender_na', 1)]], ('white', 'gender_other'): [[('white', 1), ('gender_other', 1)]], ('black', 'male'): [[('black', 1), ('male', 1)]], ('black', 'female'): [[('black', 1), ('female', 1)]], ('black', 'gender_na'): [[('black', 1), ('gender_na', 1)]], ('black', 'gender_other'): [[('black', 1), ('gender_other', 1)]], ('asian', 'male'): [[('asian', 1), ('

In [12]:
res = metrics.calc_metrics(X, y, groups, omit_demographics=omit_demographics, demographics=['country_cd_US', 'bachelor_obtained', 'white', 'black', 'asian', 'latinx', 'race_others', 'race_na', 'male', 'female', 'gender_na', 'gender_other'], protected=['country_cd_US', 'bachelor_obtained', 'white', 'black', 'asian', 'latinx', 'male', 'female'], is_gerryfair=True, iters=3)

/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


iteration: 1
most accurate classifier accuracy: 0.2132596685082873, most acc-class unfairness: 0.028302973135205994, most acc-class size 0.1558011049723757
iteration: 2
iteration: 1
most accurate classifier accuracy: 0.22320441988950276, most acc-class unfairness: 0.025029706851563745, most acc-class size 0.34917127071823206
iteration: 2
iteration: 1
most accurate classifier accuracy: 0.2185430463576159, most acc-class unfairness: 0.024478348335560424, most acc-class size 0.34216335540838855
iteration: 2
iteration: 1
most accurate classifier accuracy: 0.2229580573951435, most acc-class unfairness: 0.025190711710956493, most acc-class size 0.16887417218543047
iteration: 2
iteration: 1
most accurate classifier accuracy: 0.217439293598234, most acc-class unfairness: 0.02518455220869235, most acc-class size 0.1545253863134658
iteration: 2


/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rco

Generate a list of all subgroup data and write it to a csv

In [ ]:
csv_file = f"./data/MTC508_subgroup_data_gerryfair_with_all.csv"
# csv_file = "test.csv"

# Write the data to a CSV file
with open(csv_file, 'w', newline='') as csvfile:
    fieldnames = [
        'subgroup', 'n', 'auc_avg', 'auc_std', 'fpr_avg', 'fpr_std', 'rmse_avg', 'rmse_std']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()  # Write the header row
    for row in res:
        writer.writerow(row)

In [ ]:
demo_groups = [['country_cd_US'], ['bachelor_obtained'], ['white', 'black', 'asian', 'latinx', 'race_others', 'race_na'], ['male', 'female', 'gender_other', 'gender_na']]

In [ ]:
data = {}

data['overall'] = len(X)

for c in combos[1:]:
    masks = [X[name] == value for name, value in c]
    final_mask = pd.concat(masks, axis=1).all(axis=1)
    filter_X = X[final_mask]
    data[get_subgroup_str(c)] = len(filter_X)




In [ ]:
data

In [ ]:
df = pd.DataFrame(list(data.items()), columns=['Key', 'Count'])

# Specify the CSV file path
csv_file_path = 'data/MTC508_data_counts.csv'

# Write the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)

In [ ]:
data = []
for i in range(2,11):
    data.append(metrics.calc_metrics(X, y, groups, omit_demographics=not include_demographics, demographics=['country_cd_US', 'bachelor_obtained', 'white', 'black', 'asian', 'latinx', 'male', 'female'], is_gerryfair=True, iters=i))

In [ ]:
data

In [ ]:
fprs = []
for res in data:
    for subgroup in res:
        if subgroup['subgroup'] == 'Overall':
            fprs.append(float(subgroup['fpr_avg']))
            break

In [ ]:
fprs = np.array(fprs)
fprs.size

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

iterations = np.arange(1, 10)

other_algorithm_fpr = np.linspace(0.5, 0.5, 9) 

plt.plot(iterations, fprs, marker='o', linestyle='-', label='GerryFair')

plt.plot(iterations, other_algorithm_fpr, marker='s', linestyle='-', label='Other Algorithm')

plt.xlabel('Iterations')
plt.ylabel('False Positive Rate (FPR)')
plt.title('Comparison of Algorithms - FPR vs Iterations')

plt.legend()

plt.show()

In [ ]:
fprs

In [ ]:
demographics = ['country_cd_US', 'bachelor_obtained', 'white', 'black', 'asian', 'latinx', 'male', 'female']

In [ ]:
default_fprs = {"Overall": 0.157, "black": 0.360, "female": 0.177, "black, female": 0.537}


In [ ]:
def build_graph_iters(start, stop, step, attrs):
    data = []
    for i in range(start+1,stop+1,step):
        data.append(metrics.calc_metrics(X, y, groups, omit_demographics=not include_demographics, demographics=['country_cd_US', 'bachelor_obtained', 'white', 'black', 'asian', 'latinx', 'male', 'female'], is_gerryfair=True, iters=i))
    

    for a in attrs:
        fprs = []
        for res in data:
            
            for subgroup in res:
                if subgroup['subgroup'] == a:
                    fprs.append(float(subgroup['fpr_avg']))
                    break

        iterations = np.array(list(range(start, stop, step)))

        other_algorithm_fpr = np.linspace(default_fprs[a], default_fprs[a], iterations.size) 

        plt.plot(iterations, fprs, marker='o', linestyle='-', label='GerryFair')

        plt.plot(iterations, other_algorithm_fpr, marker='s', linestyle='-', label='Other Algorithm')

        plt.xlabel('Iterations')
        plt.ylabel('False Positive Rate (FPR)')
        plt.title('Comparison of Algorithms - FPR vs Iterations For ' + a.capitalize())

        plt.legend()

        plt.show()

In [ ]:
build_graph_iters(20,500,20,['Overall', 'black', 'female', 'black, female'])

In [ ]:
def build_graph_gamma(iters, gammas, attrs):
    results = []
    for g in gammas:
        results.append(metrics.calc_metrics(X, y, groups, omit_demographics=not include_demographics, demographics=['country_cd_US', 'bachelor_obtained', 'white', 'black', 'asian', 'latinx', 'male', 'female'], is_gerryfair=True, iters=iters+1, gamma=g))
    
    data = results
    for a in attrs:
        fprs = []
        for res in results:
            for subgroup in res:
                if subgroup['subgroup'] == a:
                    fprs.append(float(subgroup['fpr_avg']))
                    break
        
        fprs = np.array(fprs)
        gammas = np.array(gammas)

        other_algorithm_fpr = np.linspace(default_fprs[a], default_fprs[a], gammas.size) 

        plt.plot(gammas, fprs, marker='o', linestyle='-', label='GerryFair')

        plt.plot(gammas, other_algorithm_fpr, marker='s', linestyle='-', label='Other Algorithm')

        plt.xlabel('Gamma')
        plt.ylabel('False Positive Rate (FPR)')
        plt.title('Comparison of Algorithms - FPR vs Gamma For ' + a.capitalize())

        plt.legend()

        plt.show()

In [ ]:
build_graph_gamma(5, [.002, .005, .01, .02, .05, .1], ['Overall', 'black', 'female', 'black, female'])

In [ ]:
len(data)

In [ ]:
def build_graph_attrs(iters, attrs):
    results = []
    for i in range(len(demographics)):
        results.append(metrics.calc_metrics(X, y, groups, omit_demographics=not include_demographics, demographics=demographics[:i+1], is_gerryfair=True, iters=iters+1, gamma=g))
    
    for a in attrs:
        fprs = []
        for res in results:
            for subgroup in res:
                if subgroup['subgroup'] == a:
                    fprs.append(float(subgroup['fpr_avg']))
                    break
        
        fprs = np.array(fprs)
        num_attrs = np.array(list(range(1,len(demographics) + 1)))

        other_algorithm_fpr = np.linspace(default_fprs[a], default_fprs[a], len(demographics)) 

        plt.plot(num_attrs, fprs, marker='o', linestyle='-', label='GerryFair')

        plt.plot(num_attrs, other_algorithm_fpr, marker='s', linestyle='-', label='Other Algorithm')

        plt.xlabel('Number of Attributes')
        plt.ylabel('False Positive Rate (FPR)')
        plt.title('Comparison of Algorithms - FPR vs Number of Attributes For ' + a.capitalize())

        plt.legend()

        plt.show()

In [ ]:
build_graph_attrs(5, ['Overall', 'black', 'female', 'black, female'])